In [0]:
%pip install tqdm
%pip install pandas
%pip install requests

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import os
import requests
from bs4 import BeautifulSoup
from time import sleep
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd

# Create a session to handle retries
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
session.mount('http://', HTTPAdapter(max_retries=retries))
session.mount('https://', HTTPAdapter(max_retries=retries))

# Define the base URL for scraping and the local directory to save the files
scrape_url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
local_dir = "/tmp/nyc_taxi_data_2019"  # Local temporary storage in Databricks

# Create the local directory if it doesn't exist
os.makedirs(local_dir, exist_ok=True)

# Function to download a file with retries
def download_file(url, output_path):
    try:
        response = session.get(url, stream=True)
        response.raise_for_status()  # Raise an error for bad status codes
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        print(f"Successfully downloaded {output_path}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {url}: {e}")

# Scrape the website to find the correct URLs
response = session.get(scrape_url)
soup = BeautifulSoup(response.text, 'html.parser')
links = soup.find_all('a', href=True)

# Filter and download parquet files for the first three months of 2019
parquet_files = []
months = ['01', '02', '03']
for link in links:
    href = link['href']
    if any(f'2019-{month}' in href for month in months) and href.endswith('.parquet'):
        filename = href.split('/')[-1]
        url = href
        output_path = os.path.join(local_dir, filename)
        download_file(url, output_path)
        parquet_files.append(output_path)
        sleep(1)  # Sleep for a short time between requests to avoid overloading the server

print("Download completed.")

# Copy the downloaded files to DBFS
dbfs_dir = "/mnt/nyc_taxi_data_2019"  # Change to your desired DBFS path

# Create the DBFS directory if it doesn't exist
dbutils.fs.mkdirs(dbfs_dir)

# Copy files to DBFS
for local_file in parquet_files:
    dbfs_file = os.path.join(dbfs_dir, os.path.basename(local_file))
    dbutils.fs.cp(f"file:{local_file}", dbfs_file)
    print(f"Copied {local_file} to {dbfs_file}")

print("Files copied to DBFS.")

# Read and display data from each parquet file in DBFS using Spark
for local_file in parquet_files:
    dbfs_file = os.path.join(dbfs_dir, os.path.basename(local_file))
    df = spark.read.parquet(dbfs_file)
    df.show(5)

print("Data loaded and displayed.")


Successfully downloaded /tmp/nyc_taxi_data_2019/yellow_tripdata_2019-01.parquet
Successfully downloaded /tmp/nyc_taxi_data_2019/green_tripdata_2019-01.parquet
Successfully downloaded /tmp/nyc_taxi_data_2019/fhv_tripdata_2019-01.parquet
Successfully downloaded /tmp/nyc_taxi_data_2019/yellow_tripdata_2019-02.parquet
Successfully downloaded /tmp/nyc_taxi_data_2019/green_tripdata_2019-02.parquet
Successfully downloaded /tmp/nyc_taxi_data_2019/fhv_tripdata_2019-02.parquet
Successfully downloaded /tmp/nyc_taxi_data_2019/fhvhv_tripdata_2019-02.parquet
Successfully downloaded /tmp/nyc_taxi_data_2019/yellow_tripdata_2019-03.parquet
Successfully downloaded /tmp/nyc_taxi_data_2019/green_tripdata_2019-03.parquet
Successfully downloaded /tmp/nyc_taxi_data_2019/fhv_tripdata_2019-03.parquet
Successfully downloaded /tmp/nyc_taxi_data_2019/fhvhv_tripdata_2019-03.parquet
Download completed.
Copied /tmp/nyc_taxi_data_2019/yellow_tripdata_2019-01.parquet to /mnt/nyc_taxi_data_2019/yellow_tripdata_2019-01.